In [31]:
# Librerias

import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split


#python -m nltk.downloader all
from nltk.tokenize import word_tokenize
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer

**Cargar los datos y separarlos**

In [3]:
file = pd.read_excel('Train_textosODS.xlsx')
file.head(5)

,textos,ODS
0,"""Aprendizaje"" y ""educación"" se consideran sinó...",4
1,No dejar clara la naturaleza de estos riesgos ...,6
2,"Como resultado, un mayor y mejorado acceso al ...",13
3,Con el Congreso firmemente en control de la ju...,16
4,"Luego, dos secciones finales analizan las impl...",5


In [4]:
ODS = file['ODS']
textos = file['textos']

**Clase para procesar/preparar el texto**

In [47]:
class TextProcessor(BaseEstimator, TransformerMixin):
    '''
    A text processing class for text data. This transformer handles tokenization, 
    stop word removal, and stemming (optional), and transforms text data into 
    a bag-of-words representation.
    '''
    def __init__(self, token_cue = r'\w+', language = 'spanish', stemming_switch = True):
        self.language = language
        self.token_cue = token_cue
        self.stop_words_list = stopwords.words(self.language)
        self.tokenizer = RegexpTokenizer(self.token_cue)
        self.stemming_switch = stemming_switch
        self.stemmer = SnowballStemmer(self.language)
        self.vectorizer = TfidfVectorizer()       
        
    def fit(self, text, y = None):
        # Implementation to comply with pipeline API requirements
        return self

    def tokenize(self,text): #TODO
        return text.apply(lambda x: self.tokenizer.tokenize(x))
        
    def remove_stop_words(self,text): #TODO
        return text.apply(lambda x: [token for token in x if token not in self.stop_words_list])
        	
    def stemming(self,text):#TODO
        #return text.apply(lambda x: [self.stemmer.stem(token) for token in x])
        return text.apply(lambda x: [str(self.stemmer.stem(token)) for token in x])

    def BOW_transform(self,stems):        
        return self.vectorizer.fit_transform(stems.apply(lambda x: ' '.join(x)))

    def transform(self,text, y = None):
        if self.stemming_switch:
            return self.BOW_transform(self.stemming(self.remove_stop_words(self.tokenize(text)))) # With Stemming
        else: 
            return self.BOW_transform(self.remove_stop_words(self.tokenize(text))) # Without Stemming    

**Definir el Pipeline**

In [49]:
tsvd = TruncatedSVD(n_components=100)
logreg = LogisticRegression(max_iter=500)
pipeline = Pipeline([
    ('text_processor', TextProcessor()),
    ('tsvd', TruncatedSVD(n_components=100)),
    ('logreg', LogisticRegression(max_iter=500))
])

**Entrenar el modelo**

In [51]:
# Separar los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(textos, ODS, test_size = 0.2, random_state=0)
# Fit el pipeline
pipeline.fit(X_train,y_train)

Pipeline(steps=[('text_processor', TextProcessor()),
                ('tsvd', TruncatedSVD(n_components=100)),
                ('logreg', LogisticRegression(max_iter=500))])

**Resultados**

In [52]:
# predecir los datos de prueba
y_pred = pipeline.predict(X_test)
# sacar metricas
print(classification_report(y_test, y_pred))

ValueError: X has 8409 features, but TruncatedSVD is expecting 16500 features as input.

**Celdas de Prueba**

In [ ]:
# tokens = procesador.tokenize(X_train[0:20])
# no_stop = procesador.remove_stop_words(tokens)
# stems = procesador.stemming(no_stop)
# BOW = procesador.BOW_transform(stems)
# BOW = procesador.BOW_transform(stems)
# BOW[0]

In [ ]:
# svd = TruncatedSVD(n_components=100)
# BOW_reduced = svd.fit_transform(BOW)

In [ ]:
# reg = LogisticRegression(max_iter=500)
# reg.fit(BOW_reduced,y_train[:20])